!pip3 install ipywidgets

# Fútbol bayesiano

Carga de librerías

In [1]:
import os
import yaml
import pandas as pd
import matplotlib.pyplot as plt

from cmdstanpy import CmdStanModel

Se definen rutas relativas del archivo

In [2]:
CURRENT = os.getcwd()
ROOT = os.path.dirname(CURRENT)

Cargar en memoria archivo de configuración

In [3]:
with open(os.path.join(ROOT, 'config.yml'), 'r') as file_:
    config = yaml.safe_load(file_)
file_.close()

# 1. Análisis exploratorio de datos

## 1.1 Exploración tabular

Dimensiones básicas del conjunto de datos

In [4]:
# Lectura de datos en bruto
futbol_df = pd.read_csv(
    os.path.join(ROOT, config['datos']['raw'])
    ,encoding='latin-1'
)

# Generar catálogo de datos
info_ = os.path.join(ROOT, config['datos']['cat_raw'])

with open(file=info_, mode='w') as file_:
    futbol_df.info(buf=file_)
file_.close()

print(' Dims. Básicas '.center(35, '-'))
print('rows:'.ljust(10, ' ') + f'{futbol_df.shape[0]:_.0f}')
print('cols:'.ljust(10, ' ') + f'{futbol_df.shape[1]:_.0f}')

---------- Dims. Básicas ----------
rows:     23_921
cols:     44


Formatear los nombres de las columnas a *snake_case*

In [5]:
futbol_df.columns = [col_name.lower() for col_name in futbol_df.columns]

Se seleccionan únicamente los partidos registrados después del mundial del 2018 en adelante, esto es debido a que se utilizará este como periodo de entrenamiento para los modelos, para posteriormente hacer predicciones para el mundial de Qatar 2022

In [6]:
futbol_mask = (
    futbol_df
    .query('year > 2018')
    .filter(items=[
        'day'
        ,'month'
        ,'year'
        ,'tournament'
        ,'home_team'
        ,'away_team'
        ,'neutral_location'
        ,'home_team_score'
        ,'away_team_score'
        ,'home_team_fifa_rank'
        ,'away_team_fifa_rank'])
    .reset_index(drop=True)
)

futbol_mask

,day,month,year,tournament,home_team,away_team,neutral_location,home_team_score,away_team_score,home_team_fifa_rank,away_team_fifa_rank
0,2,1,2019,Friendly,Oman,Thailand,True,2,0,82,118
1,5,1,2019,AFC Asian Cup,United Arab Emirates,Bahrain,False,1,1,79,113
2,6,1,2019,AFC Asian Cup,Thailand,India,True,1,4,118,97
3,6,1,2019,AFC Asian Cup,Australia,Jordan,True,0,1,41,109
4,6,1,2019,AFC Asian Cup,Syria,Palestine,True,0,0,74,99
...,...,...,...,...,...,...,...,...,...,...,...
3016,14,6,2022,UEFA Nations League,Moldova,Andorra,False,2,1,180,153
3017,14,6,2022,UEFA Nations League,Liechtenstein,Latvia,False,0,2,192,135
3018,14,6,2022,Kirin Cup,Chile,Ghana,True,0,0,28,60
3019,14,6,2022,Kirin Cup,Japan,Tunisia,False,0,3,23,35


In [8]:
# Directorios a archivos de stan
INPUT_DIR = os.path.join(ROOT, config['modelos']['inputs'])
OUTPUT_DIR = os.path.join(ROOT, config['modelos']['outputs'])

# Rutas de modelos de stan
FILE_NAME = 'futbol_jerarquico.stan'
INPUT_ = os.path.join(INPUT_DIR, FILE_NAME)
OUTPUT_ = os.path.join(OUTPUT_DIR, FILE_NAME)


In [9]:
model_ = CmdStanModel(
    stan_file=INPUT_
    ,exe_file=OUTPUT_
    ,compile=True)